In [ ]:
######################

In [1]:
import pandas as pd
from openai import OpenAI, OpenAIError
import yaml
import time

llm_iter_counter = 0


In [2]:
# Подгрузка ключей для openai
with open (r'D:\py\keys-openrouter-free-deepseek.yml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)
api_keys = [item['key'] for item in config.get('api_keys', []) if item.get('env') == 'org-find']

In [3]:
# Импорт базы
# df = pd.read_excel(r"V:\00_Базы\04_Базы регистрации\База ГТН-РБ\база 2019-6 мес. 2025 ГТН.xlsx", sheet_name='основной массив', engine='openpyxl')
df = pd.read_excel(r"V:\00_Базы\04_Базы регистрации\orgs.xlsx", sheet_name='Sheet1', engine='openpyxl', index_col=0)

# Выходной датафрейм
df_exit = pd.DataFrame(columns=['title', 'location', 'district', 'answer'], index= df.index)

In [4]:
########## Для ГТН
# Чистка df
# 1. Убираем дубликаты по наименованию
df_m = df[['Дата регистрации','Эксплуатирующая организация','Вид деятельности эксплуатирующей организации','Область','Район']].drop_duplicates(subset='Эксплуатирующая организация')
# 2. Оставляем только те, у которых нет сферы деятельности
df_m = df_m[df_m['Вид деятельности эксплуатирующей организации'].isna()]
# 3. Заменяем кавычки
df_m['Эксплуатирующая организация'] = df_m['Эксплуатирующая организация'].str.replace('"', "'")
df_m.describe() # 3906 count

KeyError: "None of [Index(['Дата регистрации', 'Эксплуатирующая организация',\n       'Вид деятельности эксплуатирующей организации', 'Область', 'Район'],\n      dtype='object')] are in the [columns]"

In [4]:
########## Для orgs
# Чистка df
# 2. Оставляем только те, у которых нет сферы деятельности
df_m = df[df['answer'] == 'н.д.']
df_m.describe() # 1388 count

,title,location,district,answer
count,1238,1238,1238,1238
unique,1238,6,120,1
top,ЧУП ПО ОКАЗАНИЮ УСЛУГ САМВЕРС,Минская,г. Минск,н.д.
freq,1,574,240,1238


In [6]:
# Функция запроса к ллм-ке
def run_llm(api_keys, title, location, district):
  global llm_iter_counter
  
  # Цикл ключей
  api_key = api_keys[llm_iter_counter % len(api_keys)]
  # Промт вар 2 для сборки исходные + выход
  messages=[
  {
      "role": "system",
      "content": 
          "Привет! Я даю тебе юридическое наименование и локацию предприятия из Беларуси, ты возвращаешь мне текст со сферой деятельности этого предприятия"
          #"Если не достигается высокая точность, то присваивай н.д."
          "Если нет совпадений и нельзя найти сферу дейтельности, то оставлй н.д."
          "Примеры:"
          "Вход - Имя: ГОСУДАРСТВЕННОЕ ПРЕДПРИЯТИЕ 'ЗЕЛЬВЕНСКОЕ ПМС'; Область: Гродненская; Район: Зельвянский Выход - Мелиоративные предприятия'"
          "Вход - Имя: БОРИСОВСКИЙ ОПЫТНЫЙ ЛЕСХОЗ; Область: Минская; Район: Борисовский Выход - Лесхозы"
          "Вход - Имя: СЗАО 'РИНДИБЕЛ'; Область: Гродненская; Район: Волковысский Выход - Деревообработка"
          "Вход - КСУП 'БЫВАЛЬКИ'; Область: Гомельская; Район: Лоевский Выход - Сельское хозяйство"
          "Вход - Вход - ФИЛИАЛ ДЭУ-63 РУП 'МИНСКАВТОДОР-ЦЕНТР'; Область: Минская; Район: Вилейский Выход - Деятельность по эксплуатации автомобильных дорог"
  },
  {
      "role": 'user', 'content': f'Вход - Имя: {title}; Область: {location}; Район: {district}'
  } 
  ]
  # Выполнение
  client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
  )

  completion = client.chat.completions.create(
    extra_headers={
      "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
      "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
    },
    extra_body={},
    model="deepseek/deepseek-chat-v3.1:free",
    messages=messages
  )
  return completion.choices[0].message.content

In [7]:
# Подстановка значений по индексу
for i, row in df_m.iterrows():
    if i > 14540: #5708 and i <= 6514: # max(df_exit.index): (missing 5708 - 6514 )

        title = row['title'] # ['Эксплуатирующая организация'] 
        location = row['location'] # ['Область']
        district = row['district'] # ['Район']
        try:
            answer = run_llm(api_keys, title, location, district)
            llm_iter_counter +=1
            print(f'{i}: Имя: {title}; Область: {location}; Район: {district}; Answer: {answer}')
            df_exit.loc[i] = [title, location, district, answer]
            time.sleep(5)
        except OpenAIError as e:
            print(f"Error {e.body['code']} at index {i}: '{e.body['message']}'")
            break

14578: Имя: ООО 'ЭЛИТ РУУФ'; Область: Минская; Район: Смолевичский; Answer: н.д.
14583: Имя: ООО 'БИЛДАКТИВ'; Область: Брестская; Район: г. Брест; Answer: н.д.
14590: Имя: ЧП 'ПВПРОМЛЕС'; Область: Могилевская; Район: Осиповичский; Answer: н.д.
14620: Имя: ООО 'СТНОМА'; Область: Могилевская; Район: г. Бобруйск; Answer: н.д.
14622: Имя: ООО 'БИЛДГРАУНД'; Область: Гомельская; Район: г. Гомель; Answer: н.д.
14677: Имя: ФХ 'ГЕДЕОН'; Область: Брестская; Район: Столинский; Answer: н.д.
14679: Имя: ООО 'НАПРАВЛЯЮЩАЯ КОМПАНИЯ'; Область: Витебская; Район: г. Витебск; Answer: н.д.
14689: Имя: ИП 'ХУТКО Н.М.'; Область: Брестская; Район: Березовский; Answer: н.д.
14727: Имя: СПФ 'ЮНИК-АГРО' ОАО 'МИНСКИЙ МОЛОЧНЫЙ ЗАВОД №1'; Область: Минская; Район: Вилейский; Answer: Молочная промышленность
14751: Имя: ЧАСТНОЕ ПРЕДПРИЯТИЕ 'ОЛВЛАД ТРАНС'; Область: Минская; Район: Смолевичский; Answer: Транспортные услуги
14771: Имя: ЧП 'СПЕЦПАРКИНГ'; Область: Витебская; Район: Полоцкий; Answer: Автостоянки и парковки

In [14]:
llm_iter_counter +=1

In [8]:
path = r'V:\00_Базы\04_Базы регистрации\orgs2.xlsx'
df_exit.to_excel(path)